In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Define the model architecture
model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(8, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(16, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])



2024-05-24 14:14:28.661959: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [2]:
# Compile the model
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=1, verbose=1)

# Evaluate the model
_, accuracy = model.evaluate(x_test, y_test)
print("Test accuracy:", accuracy)



313/313 [==============================] - 1s 4ms/step - loss: 0.0714 - accuracy: 0.9778
Test accuracy: 0.9778000116348267


In [3]:
def representative_dataset_gen():
    for data in x_test[:100]:
        data = data.astype(np.float32)
        data = np.expand_dims(data, axis=0)
        yield [data]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()

with open("handwritten_digits_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Quantized model saved as handwritten_digits_model.tflite")

INFO:tensorflow:Assets written to: /tmp/tmp88v0oi92/assets


INFO:tensorflow:Assets written to: /tmp/tmp88v0oi92/assets


Quantized model saved as handwritten_digits_model.tflite


/home/iamme/anaconda3/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2024-05-24 14:19:08.452719: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-05-24 14:19:08.452782: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-05-24 14:19:08.454099: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp88v0oi92
2024-05-24 14:19:08.458729: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-05-24 14:19:08.458777: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp88v0oi92
2024-05-24 14:19:08.461880: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default in

In [4]:
import hls4ml
import numpy as np

# Configure HLS4ML
config = hls4ml.utils.config_from_keras_model(model, granularity='model')
config['Model']['ReuseFactor'] = 1

# Set layer-specific settings
for layer in model.layers:
    layer_config = config.get(layer.name, {})
    if isinstance(layer, tf.keras.layers.Dense):
        layer_config['Strategy'] = 'Resource'
        layer_config['Precision'] = {}
        layer_config['Precision']['weight'] = 'ap_fixed<16,6>'
        layer_config['Precision']['bias'] = 'ap_fixed<16,6>'
        layer_config['Precision']['result'] = 'ap_fixed<16,6>'
    elif isinstance(layer, tf.keras.layers.Conv2D):
        layer_config['Precision'] = {}
        layer_config['Precision']['weight'] = 'ap_fixed<16,6>'
        layer_config['Precision']['bias'] = 'ap_fixed<16,6>'
        layer_config['Precision']['result'] = 'ap_fixed<16,6>'
        layer_config['ReuseFactor'] = 1
    config[layer.name] = layer_config

# Convert the model to Verilog
hls_model = hls4ml.converters.convert_from_keras_model(model, hls_config=config)
hls_model.compile()

# Generate the Verilog code and testbench
hls_model.build(reset=False, csim=False, synth=True, vsynth=True, export=True, cosim=False)

# Prepare input data for simulation
num_samples = 100
x_test_sim = x_test[:num_samples]
y_test_sim = y_test[:num_samples]

# Run Vivado simulation and get the output
hls4ml.report.vivado_report(hls_model, x_test_sim, y_test_sim, show_outputs=True)

print("Verilog code generated and testbench simulation completed.")

Interpreting Sequential
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 28, 28, 1]], output shape: [None, 28, 28, 1]
Layer name: conv2d, layer type: Conv2D, input shapes: [[None, 28, 28, 1]], output shape: [None, 26, 26, 8]
Layer name: max_pooling2d, layer type: MaxPooling2D, input shapes: [[None, 26, 26, 8]], output shape: [None, 13, 13, 8]
Layer name: conv2d_1, layer type: Conv2D, input shapes: [[None, 13, 13, 8]], output shape: [None, 11, 11, 16]
Layer name: max_pooling2d_1, layer type: MaxPooling2D, input shapes: [[None, 11, 11, 16]], output shape: [None, 5, 5, 16]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 5, 5, 16]], output shape: [None, 400]
Layer name: dense, layer type: Dense, input shapes: [[None, 400]], output shape: [None, 64]
Layer name: dense_1, layer type: Dense, input shapes: [[None, 64]], output shape: [None, 10]
Interpreting Sequential
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 28, 28,

Exception: Vivado HLS installation not found. Make sure "vivado_hls" is on PATH.